In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce

# Data Collection and Exploration

In [ ]:
df = pd.read_csv("data-set-digital-skola-batc-10-Mizan-Amanah.xlsx - all dataset.csv")
df.head()

In [ ]:
# about data
df.info()

In [ ]:
# data summary
df.describe()

In [ ]:
# data unique
df.nunique()

## 'status_transaksi'

In [ ]:
# pesebaran jumlah data untuk 'status_transaksi'
sns.countplot(x ='status_transaksi', data = df)

## 'status_transaksi' dan 'donatur_id'

In [ ]:
# donatur_id untuk setiap jenis transaksi
df.groupby(['status_transaksi'])['donatur_id'].nunique()

In [ ]:
df[df['status_transaksi'] == 'success'].groupby(['status_transaksi'])['donatur_id'].nunique().sum()

In [ ]:
df[df['status_transaksi'] != 'success'].groupby(['status_transaksi'])['donatur_id'].nunique().sum()

In [ ]:
df['donatur_id'].nunique()

## 'tipe_transaksi'

In [ ]:
# pesebaran jumlah data untuk 'tipe_transaksi'
sns.countplot(x ='tipe_transaksi', data = df)

In [ ]:
tipe_transaksi = df.groupby('tipe_transaksi')['donasi_id'].count()
tipe_transaksi.plot(kind='pie',autopct='%.0f%%')

## 'tipe_transaksi' dan 'payment_method'

In [ ]:
df[df['tipe_transaksi'] == 'Offline'].groupby('payment_method')['donasi_id'].count().reset_index().sort_values(by='donasi_id', ascending=False)

In [ ]:
df[df['tipe_transaksi'] == 'Online'].groupby('payment_method')['donasi_id'].count().reset_index().sort_values(by='donasi_id', ascending=False)

## 'platform'

In [ ]:
df[df['tipe_transaksi'] == 'Online'].groupby('platform')['donasi_id'].count().reset_index().sort_values(by='donasi_id', ascending=False)

In [ ]:
# pesebaran jumlah data untuk 'platform'
ax = sns.countplot(x ='platform', data = df)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

## Check Missing Value

In [ ]:
df.isnull().sum()

### 'program'

In [ ]:
df_programNaN = df[df['program'].isna()]
df_programNaN.mode().loc[0]

In [ ]:
df[df['akad'] == 'Ekonomi'].mode().loc[0]

In [ ]:
df[df['program']=='Paket Sembako untuk Terdampak COVID']

In [ ]:
df_programNaN[['tgl_donasi']].agg(['min','max'])

### 'payment_method'

In [ ]:
df_paymentNaN = df[df['payment_method'].isna()]
df_paymentNaN.mode().loc[0]

In [ ]:
df_paymentNaN.groupby('platform').size()

In [ ]:
df[df['platform'].isin(['Web', 'Web Mobile'])]['payment_method'].mode().loc[0]

In [ ]:
df[df['program']=='Zakat Penghasilan'].groupby(['payment_method']).size().sort_values(ascending=False)

## Check Duplicate

In [ ]:
df[df.duplicated(keep=False)]

# Data Preprocessing

## Rename

### Rename value 'status_transaksi'

In [ ]:
df['status_transaksi'] = df['status_transaksi'].replace(['Pending', 'FAILED'], ['pending','failed'])
status_transaksi = df.groupby('status_transaksi')['donasi_id'].count()
status_transaksi.plot(kind='pie',autopct='%.0f%%')

### Rename value 'platform'

In [ ]:
df['platform'] = df['platform'].replace(['ANDROID'], ['Android'])
df['platform'] = df['platform'].replace(['WEB','web'], 'Web')
sns.countplot(y ='platform', 
                data = df,
                order = df['platform'].value_counts().index)

## Fill Missing Value

In [ ]:
df['program'].fillna('Unknown',inplace=True)
df['payment_method'].fillna('bank_transfer',inplace=True)
df.isnull().sum()

## Anomali

### Nominal dengan koma

In [ ]:
df_anomali1 = df[df['nominal'].str.contains("," , na=False)]
df_anomali1

In [ ]:
df = df[~df.isin(df_anomali1)].dropna()

### Nominal kurang dari 3 digit

In [ ]:
df_anomali2 = df.copy()
df_anomali2["nominal_length"]= df_anomali2["nominal"].astype(str).str.len()
an2 = df_anomali2[df_anomali2["nominal_length"] < 3]
an2

In [ ]:
df = df[~df.isin(an2)].dropna()

### Nominal tidak wajar

In [ ]:
df_anomali3 = df.copy()
df_anomali3["nominal_length"]= df_anomali3["nominal"].astype(str).str.len()
an3 = df_anomali3[df_anomali3["nominal_length"] >= 10]
an3

In [ ]:
df = df[~df.isin(an3)].dropna()

# Data Transformation (Feature Engineering)

## Mengubah Data Type

In [ ]:
df['tgl_donasi'] = pd.to_datetime(df['tgl_donasi'])
df['nominal'] = df['nominal'].astype('int64')
df.info()

## Menambah kolom tahun dan bulan

In [ ]:
df['tahun'] = df['tgl_donasi'].dt.year
df['bulan'] = df['tgl_donasi'].dt.month
df.head()

## Menambah kolom jumlah donasi untuk setiap donatur

In [ ]:
df['jmlh_donasi'] = df['donatur_id'].map(df['donatur_id'].value_counts())
df.head()

## Data Filter untuk 'tipe_transaksi' success dan not succes

In [ ]:
df_success = df.copy()
df_success = df_success[df_success['status_transaksi'] == 'success']
df_success.head()

In [ ]:
df_not_success = df.copy()
df_not_success = df_not_success[df_not_success['status_transaksi'] != 'success']
df_not_success.head()

## Membuat data pertahun dan menambah kolom jumlah transaksi donasi pertahun

In [ ]:
df.tahun.unique()

In [ ]:
tahun = df.tahun.unique()
d = {}
for x in tahun:
    d["th{0}".format(x)] = df.loc[df['tahun']== x].copy()
    d["th{0}".format(x)]['donasi_tahunini'] = d["th{0}".format(x)]['donatur_id'].map(d["th{0}".format(x)]['donatur_id'].value_counts())

In [ ]:
display(d['th2019'].head())

In [ ]:
df_process = pd.concat([d['th2019'],d['th2020'],d['th2021'],d['th2022'],d['th2023']], ignore_index=True)
df_process['tgl_donasi'].agg(['min','max'])

# Analysis

## Jumlah donatur belum berhasil transaksi

In [ ]:
df_process.groupby('status_transaksi')['donatur_id'].nunique().reset_index()

In [ ]:
sns.countplot(data=df_process, x='status_transaksi')

In [ ]:
# donatur yang transaksinya pernah gagal
donatur_successfailed = np.intersect1d(df_success.donatur_id, df_not_success.donatur_id)
donatur_successfailed.size

In [ ]:
# donatur yang transaksinya selalu berhasil
donatur_success = np.setxor1d(df_success.donatur_id, donatur_successfailed)
donatur_success.size

In [ ]:
# donatur yang transaksinya belum pernah berhasil
donatur_failed = np.setxor1d(df_not_success.donatur_id, donatur_successfailed)
donatur_failed.size

In [ ]:
donatur_failed

In [ ]:
df_donatur_failed = df_process.loc[df_process['donatur_id'].isin(donatur_failed)]
df_donatur_failed

### Results

In [ ]:
transaksi_donatur = pd.DataFrame({'jumlah_donatur': [donatur_success.size, donatur_successfailed.size , donatur_failed.size]},
                                index=['donatur_success', 'donatur_successfailed', 'donatur_failed'])
transaksi_donatur

In [ ]:
colors = ['#99ff99','#66b3ff','#ff9999']
transaksi_donatur.plot(kind='pie',y='jumlah_donatur', colors=colors, autopct='%.0f%%')

In [ ]:
df_donatur_failed.groupby(['tahun'])['donasi_id'].count()

In [ ]:
df_donatur_failed.groupby(['platform'])['donasi_id'].count()

In [ ]:
df_donatur_failed.groupby(['tipe_transaksi'])['donasi_id'].count()

In [ ]:
df_process.groupby(['platform'])['donasi_id'].count()

### Insight

`Donasi dengan interaksi secara langsung baik dilakukan pada Offline ataupun secara WA terbukti tidak mengalami kegagalan, sedangkan melalui platform online terdapat banyak transaksi donasi yang gagal. Penyebab gagalnya transaksi/donasi online bisa jadi terdapat error pada sistem ataupun upaya dari donatur sendiri. Upaya yang dilakukan bisa dengan selalu memantau sistem onlinenya ataupun melakukan perbaikan pada sistem online`


## Donatur pertama kali donasi

In [ ]:
df_first_donation = df_process.copy()
df_first_donation.drop_duplicates(subset='donatur_id', keep='first', inplace=True)

In [ ]:
donatur_baru_pertahun = df_first_donation.groupby('tahun').donatur_id.count().reset_index()
donatur_baru_pertahun

In [ ]:
sns.countplot(x ='tahun', data=df_first_donation)

In [ ]:
donatur_baru_perbulan = df_first_donation.groupby(['tahun','bulan'])['donatur_id'].count().reset_index()

In [ ]:
donatur_baru_perbulan_pivot = donatur_baru_perbulan.pivot(index='bulan', columns='tahun', values='donatur_id')
donatur_baru_perbulan_pivot

### Results

In [ ]:
# Donatur pertama kali donasi = Bertambahnya jumlah donatur setiap tahun
donatur_baru_pertahun['tahun'] = donatur_baru_pertahun.tahun.astype(str)
sns.lineplot(data=donatur_baru_pertahun, x="tahun", y="donatur_id", marker='.', markerfacecolor='limegreen', markersize=20).set(title='Jumlah Donatur Baru', xlabel='Tahun', ylabel='Donatur')

In [ ]:
donatur_baru_pertahun['growth'] = np.round(donatur_baru_pertahun['donatur_id'].pct_change()*100,2)
#donatur_baru_pertahun.drop(donatur_baru_pertahun.iloc[0])
donatur_baru_pertahun

In [ ]:
print('growth rate 2019-2022: ',np.mean([27.28,-13.92,17.78]))

In [ ]:
donatur_baru_perbulan_pivot.plot(kind='bar')

In [ ]:
donatur_baru_platform = df_first_donation.groupby('platform')['donatur_id'].count().reset_index().sort_values(by='donatur_id',ascending=False)
donatur_baru_platform

In [ ]:
ax = sns.countplot(x='platform', data=df_first_donation)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

### Insights

`Berdasarkan data jumlah donatur pertama kali donasi atau disebut donatur baru setiap tahunnya mengalami rata-rata kenaikan 10.38%% dengan puncaknya pada bulan 4-5 yaitu bulan2 ramadhan dengan dominasi transaksi offline. Mizan amanah memiliki banyak ruang untuk improve melalui sistem online dengan memanfaatkan pada bulan2 ramadhan`

## Jumlah Donatur 1x transaksi pertahun

In [ ]:
transaksi_1x_pertahun = df_process[df_process['donasi_tahunini']==1].groupby('tahun')['donatur_id'].nunique().reset_index()
transaksi_1x_pertahun.columns = transaksi_1x_pertahun.columns.str.replace('donatur_id', 'transaksi_1x')
transaksi_1x_pertahun

In [ ]:
transaksi_1x_pertahun['transaksi_1x'].sum()

In [ ]:
print ("{:.0%}".format(transaksi_1x_pertahun['transaksi_1x'].sum()/df_process['donatur_id'].nunique()))

In [ ]:
transaksi_1x_perbulan = df_process[df_process['donasi_tahunini']==1].groupby(['tahun','bulan'])['donatur_id'].nunique().reset_index()
transaksi_1x_perbulan.columns = transaksi_1x_perbulan.columns.str.replace('donatur_id', 'transaksi_1x')

In [ ]:
transaksi_1x_perbulan_pivot = transaksi_1x_perbulan.pivot(index='bulan', columns='tahun', values='transaksi_1x')
transaksi_1x_perbulan_pivot

### Results

In [ ]:
sns.lineplot(x='tahun', y='transaksi_1x', data=transaksi_1x_pertahun)

In [ ]:
transaksi_1x_pertahun['growth'] = np.round(transaksi_1x_pertahun['transaksi_1x'].pct_change()*100,2)
transaksi_1x_pertahun

In [ ]:
transaksi_1x_perbulan_pivot.plot(kind='bar')

In [ ]:
transaksi_1x_2022 = df_process[(df_process['donasi_tahunini']==1) &
           (df_process['tahun']==2022)].groupby('bulan')['donatur_id'].nunique().reset_index()
transaksi_1x_2022['rate'] = np.round(transaksi_1x_2022['donatur_id'].pct_change()*100,2)
transaksi_1x_2022

In [ ]:
transaksi_1x_program = df_process[(df_process['donasi_tahunini']==1)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index()
transaksi_1x_program['rate'] = (transaksi_1x_program['donatur_id']/transaksi_1x_program['donatur_id'].sum())*100
transaksi_1x_program.iloc[:10]

In [ ]:
df_process[(df_process['donasi_tahunini']==1) &
           (df_process['tahun']==2022)&
           (df_process['bulan']==4)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).iloc[:10]

In [ ]:
df_process[(df_process['program']=='Berbagi Baju Lebaran')].groupby(['tahun','bulan'])['donatur_id'].nunique()

### Insights
`Setiap tahunnya jumlah donatur 1x transaksi terus mengalami peningkatan dengan pola yang sama yaitu bulan 4-5 (bulan ramadhan) merupakan transaksi yang paling banyak dan tahun 2022 bulan 4 merupakan kenaikan paling signifikan sebesar 400.71%. Mizan amanah bisa memanfaatkan program ramadhan unggulannya seperti pada tahun 2022 bulan 4 untuk mencari donatur melakukan donasi`

## Jumlah Donatur 2x transaksi pertahun

In [ ]:
#donatur konsisten 2x pertahun
val_2x = reduce(np.intersect1d, (d['th2019'][d['th2019']['donasi_tahunini'] >= 2].donatur_id, 
                              d['th2020'][d['th2020']['donasi_tahunini'] >= 2].donatur_id, 
                              d['th2021'][d['th2021']['donasi_tahunini'] >= 2].donatur_id,
                              d['th2022'][d['th2022']['donasi_tahunini'] >= 2].donatur_id,
                              d['th2023'][d['th2023']['donasi_tahunini'] >= 2].donatur_id,))
val_2x.size

In [ ]:
transaksi_2x_pertahun = df_process[df_process['donasi_tahunini']==2].groupby('tahun')['donatur_id'].nunique().reset_index()
transaksi_2x_pertahun.columns = transaksi_2x_pertahun.columns.str.replace('donatur_id', 'transaksi_2x')
transaksi_2x_pertahun

In [ ]:
transaksi_2x_perbulan = df_process[df_process['donasi_tahunini']==2].groupby(['tahun','bulan'])['donatur_id'].nunique().reset_index()
transaksi_2x_perbulan.columns = transaksi_2x_perbulan.columns.str.replace('donatur_id', 'transaksi_1x')

In [ ]:
transaksi_2x_perbulan_pivot = transaksi_2x_perbulan.pivot(index='bulan', columns='tahun', values='transaksi_1x')
transaksi_2x_perbulan_pivot

### Results

In [ ]:
sns.lineplot(x='tahun', y='transaksi_2x', data=transaksi_2x_pertahun)

In [ ]:
transaksi_2x_pertahun['growth'] = np.round(transaksi_2x_pertahun['transaksi_2x'].pct_change()*100,2)
transaksi_2x_pertahun

In [ ]:
transaksi_2x_pertahun['transaksi_2x'].sum()

In [ ]:
print ("{:.0%}".format(transaksi_2x_pertahun['transaksi_2x'].sum()/df_process['donatur_id'].nunique()))

In [ ]:
transaksi_2x_perbulan_pivot.plot(kind='bar')

In [ ]:
transaksi_2x_program = df_process[(df_process['donasi_tahunini']==2)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index()
transaksi_2x_program['rate'] = (transaksi_2x_program['donatur_id']/transaksi_2x_program['donatur_id'].sum())*100
transaksi_2x_program.iloc[:10]

In [ ]:
df_process[(df_process['donasi_tahunini']==2) &
           (df_process['bulan']==5)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).iloc[:10]

In [ ]:
df_process[(df_process['donasi_tahunini']==2)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).iloc[:10]

### Insights
`Sama seperti transaksi 1x, Mizan amanah bisa berfokus pada bulan 4-5 untuk menambah transaksi 2x dengan program ramadhan, zakat, dan sedekah yatim` 

## Donatur habit 12x pertahun

In [ ]:
# donatur yg konsisten 12x transaksi setiap tahunnya
val_12x = reduce(np.intersect1d, (d['th2019'][d['th2019']['donasi_tahunini'] >= 12].donatur_id, 
                              d['th2020'][d['th2020']['donasi_tahunini'] >= 12].donatur_id, 
                              d['th2021'][d['th2021']['donasi_tahunini'] >= 12].donatur_id,
                              d['th2022'][d['th2022']['donasi_tahunini'] >= 12].donatur_id,
                              d['th2023'][d['th2023']['donasi_tahunini'] >= 12].donatur_id,))
val_12x

In [ ]:
df_process[df_process['donatur_id'] == '7a711fe2bd474cffcd660c64b05e4a30']

In [ ]:
transaksi_12x_pertahun = df_process[df_process['donasi_tahunini']>=12].groupby('tahun')['donatur_id'].nunique().reset_index()
transaksi_12x_pertahun.columns = transaksi_12x_pertahun.columns.str.replace('donatur_id', 'transaksi_12x')
transaksi_12x_pertahun

In [ ]:
transaksi_12x_perbulan = df_process[df_process['donasi_tahunini']>=12].groupby(['tahun','bulan'])['donatur_id'].nunique().reset_index()
transaksi_12x_perbulan.columns = transaksi_12x_perbulan.columns.str.replace('donatur_id', 'transaksi_12x')

In [ ]:
transaksi_12x_perbulan_pivot = transaksi_12x_perbulan.pivot(index='bulan', columns='tahun', values='transaksi_12x')
transaksi_12x_perbulan_pivot

### Results

In [ ]:
sns.lineplot(x='tahun', y='transaksi_12x', data=transaksi_12x_pertahun)

In [ ]:
transaksi_12x_pertahun['growth'] = np.round(transaksi_12x_pertahun['transaksi_12x'].pct_change()*100,2)
transaksi_12x_pertahun

In [ ]:
transaksi_12x_pertahun['transaksi_12x'].sum()

In [ ]:
print ("{:.0%}".format(transaksi_12x_pertahun['transaksi_12x'].sum()/df_process['donatur_id'].nunique()))

In [ ]:
transaksi_12x_perbulan_pivot.plot(kind='bar')

In [ ]:
transaksi_12x_program = df_process[(df_process['donasi_tahunini']>=12)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index()
transaksi_12x_program['rate'] = (transaksi_12x_program['donatur_id']/transaksi_12x_program['donatur_id'].sum())*100
transaksi_12x_program.iloc[:10]

In [ ]:
df_process[df_process['donasi_tahunini']>=12].groupby(['bulan'])['donatur_id'].nunique().plot(kind='bar')

In [ ]:
df_process[(df_process['donasi_tahunini']>=12) &
           (df_process['tahun']==2019)].groupby('akad')['donatur_id'].nunique().sort_values(ascending=False).reset_index().iloc[:10]

In [ ]:
df_process[(df_process['donasi_tahunini']>=12) &
           (df_process['tahun']==2020)].groupby('akad')['donatur_id'].nunique().sort_values(ascending=False).reset_index().iloc[:10]

In [ ]:
df_process[(df_process['donasi_tahunini']>=12)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index().iloc[:10]

### Insights

`Pertumbuhan signifikan transaksi 12x dari 2019 ke 2020 ternyata pada tahun itu baru dibuat akad/program ramadhan sehingga momen pada bulan 4-5 (bulan ramadhan) adalah momen yang paling tepat untuk menambah donatur atau transaksi. Dari pola data perbulan pun terlihat setiap bulannnya stabil, terlihat dari program bahwa donatur 12x ini selain program yatim dan lebaran juga banyak di program zakat dan sedekah.`

## Donatur 3x > donasi < 12x pertahun

In [ ]:
# donatur konsisten setiap tahunnya dengan rentang 3x > donasi < 12x
val_3x12x = reduce(np.intersect1d, (d['th2019'][(d['th2019']['donasi_tahunini'] >= 3) & (d['th2019']['donasi_tahunini'] < 12)].donatur_id, 
                              d['th2020'][(d['th2020']['donasi_tahunini'] >= 3) & (d['th2020']['donasi_tahunini'] < 12)].donatur_id, 
                              d['th2021'][(d['th2021']['donasi_tahunini'] >= 3) & (d['th2021']['donasi_tahunini'] < 12)].donatur_id,
                              d['th2022'][(d['th2022']['donasi_tahunini'] >= 3) & (d['th2022']['donasi_tahunini'] < 12)].donatur_id,
                              d['th2023'][(d['th2023']['donasi_tahunini'] >= 3) & (d['th2023']['donasi_tahunini'] < 12)].donatur_id))
val_3x12x

In [ ]:
transaksi_3x12x_pertahun = df_process[(df_process['donasi_tahunini']>=3) & (df_process['donasi_tahunini']<12)].groupby('tahun')['donatur_id'].nunique().reset_index()
transaksi_3x12x_pertahun.columns = transaksi_3x12x_pertahun.columns.str.replace('donatur_id', 'transaksi_3x12x')
transaksi_3x12x_pertahun

In [ ]:
transaksi_3x12x_perbulan = df_process[(df_process['donasi_tahunini']>=3) & (df_process['donasi_tahunini']<12)].groupby(['tahun','bulan'])['donatur_id'].nunique().reset_index()
transaksi_3x12x_perbulan.columns = transaksi_12x_perbulan.columns.str.replace('donatur_id', 'transaksi_3x12x')

In [ ]:
transaksi_3x12x_perbulan_pivot = transaksi_3x12x_perbulan.pivot(index='bulan', columns='tahun', values='transaksi_12x')
transaksi_3x12x_perbulan_pivot

### Results

In [ ]:
sns.lineplot(x='tahun',y='transaksi_3x12x',data=transaksi_3x12x_pertahun)

In [ ]:
transaksi_3x12x_pertahun['growth'] = np.round(transaksi_3x12x_pertahun['transaksi_3x12x'].pct_change()*100,2)
transaksi_3x12x_pertahun

In [ ]:
transaksi_3x12x_pertahun['transaksi_3x12x'].sum()

In [ ]:
print ("{:.0%}".format(transaksi_3x12x_pertahun['transaksi_3x12x'].sum()/df_process['donatur_id'].nunique()))

In [ ]:
transaksi_3x12x_perbulan_pivot.plot(kind='bar')

In [ ]:
df_process[(df_process['donasi_tahunini']>=3) & (df_process['donasi_tahunini']<12)].groupby(['bulan'])['donatur_id'].nunique().plot(kind='bar')

In [ ]:
transaksi_3x12x_program = df_process[(df_process['donasi_tahunini']>=3) &
           (df_process['donasi_tahunini']<12)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index()
transaksi_3x12x_program['rate'] = (transaksi_3x12x_program['donatur_id']/transaksi_3x12x_program['donatur_id'].sum())*100
transaksi_3x12x_program.iloc[:10]

### Insights 

`Bulan 4-5 (bulan ramadhan) memang sebagai puncak donasi oleh para donatur 3x-11x, tetapi pola data perbulan untuk setiap bulannya sudah mulai berimbang. Terlihat dari program transaksi 3x-11x terdapat program tunaikan zakat penghasilan`

## Summary

In [ ]:
jumlah_transaksi = transaksi_1x_pertahun.merge(transaksi_2x_pertahun, on='tahun').merge(transaksi_3x12x_pertahun, on='tahun').merge(transaksi_12x_pertahun, on='tahun')
jumlah_transaksi = jumlah_transaksi.drop(columns=['growth_x', 'growth_y'])
jumlah_transaksi

In [ ]:
sns.barplot(data=jumlah_transaksi.melt(id_vars='tahun',
                                  value_name='donatur', var_name='jumlah_transaksi'),
            x='tahun', y='donatur', hue='jumlah_transaksi')

In [ ]:

sns.barplot(data=jumlah_transaksi.melt(id_vars='tahun',
                                  value_name='donatur', var_name='jumlah_transaksi'),
            x='tahun', y='donatur', hue='jumlah_transaksi')

## Donatur Reguler pertahun

In [ ]:
val_reguler = reduce(np.intersect1d, (d['th2019'].donatur_id, 
                              d['th2020'].donatur_id, 
                              d['th2021'].donatur_id,
                              d['th2022'].donatur_id,
                              d['th2023'].donatur_id))
val_reguler.size

In [ ]:
val_reguler2 = reduce(np.intersect1d, (d['th2019'].donatur_id, 
                              d['th2020'].donatur_id, 
                              d['th2021'].donatur_id,
                              d['th2022'].donatur_id))
val_reguler2.size

In [ ]:
df_donatur_reguler = df_process.loc[df_process['donatur_id'].isin(val_reguler)]
df_donatur_reguler

In [ ]:
df_donatur_reguler['donatur_id'].nunique()

### Results

In [ ]:
donasidonatur_reguler_perbulan = df_donatur_reguler.groupby(['tahun','bulan'])['donasi_id'].count().reset_index()
donasidonatur_reguler_perbulan_pivot = donasidonatur_reguler_perbulan.pivot(index='bulan', columns='tahun', values='donasi_id')
donasidonatur_reguler_perbulan_pivot

In [ ]:
df_donatur_reguler['donatur_id'].nunique()

In [ ]:
print ("{:.2%}".format(df_donatur_reguler['donatur_id'].nunique()/df_process['donatur_id'].nunique()))

In [ ]:
donasidonatur_reguler_perbulan_pivot.plot(kind='bar')

In [ ]:
donatur_reguler_perbulan = df_donatur_reguler.groupby(['tahun','bulan'])['donatur_id'].nunique().reset_index()
donatur_reguler_perbulan_pivot = donatur_reguler_perbulan.pivot(index='bulan', columns='tahun', values='donatur_id')
donatur_reguler_perbulan_pivot

In [ ]:
donatur_reguler_perbulan_pivot.plot(kind='bar')

In [ ]:
df_donatur_reguler.groupby(['bulan'])['donatur_id'].nunique().reset_index().plot()

In [ ]:
reguler_2019 = df_donatur_reguler[(df_donatur_reguler['tahun']==2019)].groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index()
reguler_2019['rate'] = (reguler_2019['donatur_id']/reguler_2019['donatur_id'].sum())*100
reguler_2019.iloc[:10]

In [ ]:
reguler = df_donatur_reguler.groupby('program')['donatur_id'].nunique().sort_values(ascending=False).reset_index()
reguler['rate'] = (reguler['donatur_id']/reguler['donatur_id'].sum())*100
reguler.iloc[:10]

In [ ]:
donatur_reguler_akad = df_donatur_reguler.groupby('akad')['donatur_id'].nunique().sort_values(ascending=False).iloc[:10].reset_index()
donatur_reguler_akad['rate'] = np.round(donatur_reguler_akad['donatur_id']/df_donatur_reguler['donatur_id'].nunique()*100,2)
donatur_reguler_akad

In [ ]:
donatur_all_akad = df_process.groupby('akad')['donatur_id'].nunique().sort_values(ascending=False).iloc[:10].reset_index()
donatur_all_akad['rate'] = np.round(donatur_all_akad['donatur_id']/df_process['donatur_id'].nunique()*100,2)
donatur_all_akad

In [ ]:
df_process.groupby('program')['donatur_id'].nunique().sort_values(ascending=False).iloc[:10].reset_index()

In [ ]:
df_donatur_reguler.groupby('donatur_id').agg({'donasi_id':'count',
                                              'nominal':'sum'}).sort_values(by='donasi_id',ascending=False)

### Insights

`Donatur reguler juga tidak lepas dari momen ramadhan dimana donasi banyak dilakukan. Namun ketika ditelusuri melalui program donatur reguler banyak melakukan donasi pada program yatim dan zakat, Pada akad Terlihat dari 38% donatur reguler melakukan donasi pada akad zakat profesi dibandingkan keseluruhan donatur hanya 7%.`

`Dapat disimpulkan untuk menambah donatur reguler yang konsisten berdonasi setiap tahun bisa berfokus pada program yatim dan zakat.`

# DF for dashboard

In [ ]:
# tambah kolom first donasi
val_first = df_first_donation['donasi_id']
df_process['first_donasi'] = np.where(df_process['donasi_id'].isin(val_first), 'yes', 'no')

# tambah kolom donatur reguler
df_process['reguler'] = np.where(df_process['donatur_id'].isin(val_reguler), 'yes', 'no')


In [ ]:
# tambah kolom kategori
conditions = [df_process['donatur_id'].isin(donatur_success), df_process['donatur_id'].isin(donatur_successfailed), df_process['donatur_id'].isin(donatur_failed)]
choices = ['success', 'success failed', 'failed']
df_process['kategori'] = np.select(conditions, choices)

In [ ]:
# to csv
df_process.to_csv('dataset_RetentionProcess_2.csv', index=False)

In [ ]:
df_process[df_process['status_transaksi']=='success']['nominal'].sum()

# Save Data

In [ ]:
# from openpyxl import load_workbook

# path = r"dataset_RetentionAnalysis.xlsx"

# book = load_workbook(path)
# writer = pd.ExcelWriter(path, engine = 'openpyxl')
# writer.book = book

# df_tahunan.to_excel(writer, sheet_name = 'df')
# df_success.to_excel(writer, sheet_name = 'df_success')
# df_not_success.to_excel(writer, sheet_name = 'df_not_success')
# df_first_donation.to_excel(writer, sheet_name = 'df_first_donation')
# df_donatur_reguler.to_excel(writer, sheet_name = 'df_donatur_reguler')
# transaksi_donatur.to_excel(writer, sheet_name = 'transaksi_donatur')
# jumlah_donatur_baru.to_excel(writer, sheet_name = 'jumlah_donatur_baru')
# jumlah_transaksi.to_excel(writer, sheet_name = 'jumlah_transaksi')


# writer.close()